# Projects in data science, Project: Skin lesions

# Part 1 extracting features

##### Imports

In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage import segmentation, color
import csv
from skimage.feature import graycomatrix, graycoprops
from skimage import io, color, img_as_ubyte
import skimage.util as util



##### File paths, please update before running

In [29]:
Lesion_images_folder="C:\\Users\\elias\\Desktop\\Projects in data science\\Final project\\Lesions_images"
Lesion_masks_folder="C:\\Users\\elias\\Desktop\\Projects in data science\\Final project\\Lesions_masks"
Metadata_path="C:\\Users\\elias\\Desktop\\Projects in data science\\Final project\\metadata.csv"

# Functions for extracting features/attributes

### Asymmetri

In [3]:
def measure_symmetry(img_path):
    # Load image
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

    # Calculate area total
    area_total = np.count_nonzero(img)

    # Calculate horizontal flip and difference
    horizontal_flip = np.fliplr(img)
    diff_horizontal = img * ~horizontal_flip
    diff_horizontal_area = np.count_nonzero(diff_horizontal)

    # Calculate vertical flip and difference
    vertical_flip = np.flipud(img)
    diff_vertical = img * ~vertical_flip
    diff_vertical_area = np.count_nonzero(diff_vertical)

    # Calculate asymmetric index
    if area_total==0:
        return 0
    else:   
        asymm_idx = 0.5 * ((diff_horizontal_area / area_total) + (diff_vertical_area / area_total))
        return asymm_idx



### Colour

In [4]:
def analyze_and_visualize_segmentation(image_path, mask_path, lst_compactness):
    # Load RGB image and mask
    rgb_img = plt.imread(image_path)[:,:,:3]
    mask = plt.imread(mask_path)

    # Replace pixels within the lesion area with the average color of the lesion
    img_avg_lesion = rgb_img.copy()
    for i in range(3):
        channel = img_avg_lesion[:,:,i]
        mean = np.mean(channel[mask.astype(bool)])
        channel[mask.astype(bool)] = mean
        img_avg_lesion[:,:,i] = channel

    # Crop the lesion area from the original image
    lesion_coords = np.where(mask != 0)
    min_x, max_x = min(lesion_coords[0]), max(lesion_coords[0])
    min_y, max_y = min(lesion_coords[1]), max(lesion_coords[1])
    cropped_lesion = rgb_img[min_x:max_x, min_y:max_y]

    # Initialize lists to store average colors and standard deviations
    avg_colors = []
    std_devs = []

    # Perform SLIC segmentation and calculate average colors for the lesion
    for c in lst_compactness:
        labels = segmentation.slic(cropped_lesion, compactness=c, n_segments=30, sigma=3, start_label=1)
        avg_color_lesion = np.mean(cropped_lesion[labels != 0], axis=0)
        std_dev_lesion = np.std(cropped_lesion[labels != 0], axis=0)
        avg_colors.append(avg_color_lesion)
        std_devs.append(std_dev_lesion)

    # Calculate the average RGB values and standard deviations
    avg_avg_colors = np.mean(avg_colors, axis=0)
    avg_std_devs = np.mean(std_devs, axis=0)

    return list(avg_avg_colors)+list(avg_std_devs)


### Blue-White veil

In [5]:
def classify_pixel_as_veil(rgb_img):
    veil_count = 0
    
    for i in range(rgb_img.shape[0]):
        for j in range(rgb_img.shape[1]):
            if len(rgb_img[i, j]) == 3:  # Check if pixel has RGB values
                R, G, B = rgb_img[i, j]
                Lum = R + G + B
                nB = B / Lum

                if nB >= 0.3 and 0.6 <= Lum <= 2:
                    veil_count += 1
                
    return veil_count

def analyze_and_count_veil(image_path, mask_path):
    # Load RGB image and mask
    rgb_img = plt.imread(image_path).astype(np.float32)  # Ensure float32 type
    mask = plt.imread(mask_path)
    
    # Apply mask to RGB image
    masked_rgb_img = rgb_img * mask[:, :, np.newaxis]
    
    # Classify each pixel as veil or non-veil and count veil pixels
    veil_count = classify_pixel_as_veil(masked_rgb_img)
    
    return veil_count


### Haralick texture

In [6]:
def compute_haralick_texture_features(image_path, mask_path):
    image = io.imread(image_path)
    mask = io.imread(mask_path)
    
    # Remove alpha channel if present
    if image.shape[2] == 4:
        image = image[:, :, :3]  # Remove the alpha channel
    
    # Convert RGB image to grayscale
    gray_image = color.rgb2gray(image)
    # Convert grayscale image to unsigned integer type
    gray_image_uint = util.img_as_ubyte(gray_image)
    
    # Apply mask to image
    masked_image = image * mask[:, :, np.newaxis]
    
    # Compute gray-level co-occurrence matrix (GLCM)
    distances = [1]  # distance between pixels
    angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]  # angles for texture measurements
    glcm = graycomatrix(gray_image_uint, distances=distances, angles=angles, levels=256, symmetric=True, normed=True)
    
    # Compute Haralick texture features
    contrast = graycoprops(glcm, 'contrast').ravel().mean()
    dissimilarity = graycoprops(glcm, 'dissimilarity').ravel().mean()
    homogeneity = graycoprops(glcm, 'homogeneity').ravel().mean()
    energy = graycoprops(glcm, 'energy').ravel().mean()
    correlation = graycoprops(glcm, 'correlation').ravel().mean()
    
    return [contrast, dissimilarity, homogeneity, energy, correlation]




In [7]:
# Compute texture features
image_path = "C:\\Users\\elias\\Desktop\\Projects in data science\\Final project\\Lesions_images\\PAT_1064_272_668.png"
mask_path =  "C:\\Users\\elias\\Desktop\\Projects in data science\\Final project\\Lesions_masks\\PAT_1064_272_668_mask.png"

contrast, dissimilarity, homogeneity, energy, correlation = compute_haralick_texture_features(image_path, mask_path)

print("Contrast:", contrast)
print("Dissimilarity:", dissimilarity)
print("Homogeneity:", homogeneity)
print("Energy:", energy)
print("Correlation:", correlation)

Contrast: 65.63454833745564
Dissimilarity: 5.269616317150561
Homogeneity: 0.2171990184897258
Energy: 0.021375145329951103
Correlation: 0.969490575012103


### Diagnostic from metadata

In [8]:
def get_diagnostic(img_name):
    
    search_value=img_name+".png"
    diagnostic = None
    
    with open(Metadata_path, 'r') as csv_file:
        reader = csv.DictReader(csv_file, delimiter=',')
        
        for row in reader:
            if row["img_id"] == search_value:
                diagnostic = row["diagnostic"]
                break  # Stop searching once the value is found
    if diagnostic in ["BCC","SCC","MEL"]:
        return(1)
    else:
        return(0)


# Extracting and saving feature values

In [33]:
Asymmetry_results={}
# Loop through each image in the folder
for filename in os.listdir(Lesion_masks_folder):
    if filename.endswith('.jpg') or filename.endswith('.png'):  # Adjust file extensions as needed
        img_path = os.path.join(Lesion_masks_folder, filename)
        result = measure_symmetry(img_path)
        ID=filename.replace("_mask.png","");
        Asymmetry_results[ID]=result
#Asymmetry_results      


In [19]:
Colour_results = {}

# Loop through each image in the folder
for filename in os.listdir(Lesion_images_folder):
    if filename.endswith('.jpg') or filename.endswith('.png'):  # Adjust file extensions as needed
        img_path = os.path.join(Lesion_images_folder, filename)
        
        # Find corresponding mask
        corresponding_mask = os.path.join(Lesion_masks_folder, os.path.basename(os.path.splitext(img_path)[0]) + "_mask.png")
        
        # Analyze and visualize segmentation
        result = analyze_and_visualize_segmentation(img_path, corresponding_mask, [7])
        
        # Append results to Colour_results list
        ID=filename.replace(".png","");
        Colour_results[ID]=result

#print(Colour_results)


Blue-White tager nogle minutter.

In [11]:
Blue_white_veil_results = {}

# Loop through each image in the folder
for filename in os.listdir(Lesion_images_folder):
    if filename.endswith('.jpg') or filename.endswith('.png'):  # Adjust file extensions as needed
        img_path = os.path.join(Lesion_images_folder, filename)
        
        # Find corresponding mask
        corresponding_mask = os.path.join(Lesion_masks_folder, os.path.basename(os.path.splitext(img_path)[0]) + "_mask.png")
        
        # cound blue white veils pixels
        result = analyze_and_count_veil(img_path, corresponding_mask)
        
        # Append results to Blue_white_veil_results list
        ID=filename.replace(".png","");
        Blue_white_veil_results[ID]=result



C:\Users\elias\AppData\Local\Temp\ipykernel_17676\158654455.py:9: RuntimeWarning: invalid value encountered in scalar divide
  nB = B / Lum


In [12]:
haralick_results = {}

# Loop through each image in the folder
for filename in os.listdir(Lesion_images_folder):
    if filename.endswith('.jpg') or filename.endswith('.png'):  # Adjust file extensions as needed
        img_path = os.path.join(Lesion_images_folder, filename)
        
        # Find corresponding mask
        corresponding_mask = os.path.join(Lesion_masks_folder, os.path.basename(os.path.splitext(img_path)[0]) + "_mask.png")
        
        # compute haralick
        result = compute_haralick_texture_features(img_path, corresponding_mask)
        
        # Append results to haralick results
        ID=filename.replace(".png","");
        haralick_results[ID]=result


# Creating CSV file and and headers

### Create file and add headers

In [20]:
# Get the current working directory
current_directory = os.getcwd()
# Define the filename for the CSV file
csv_filename = "Attributes.csv"
# Construct the full path to the CSV file
csv_filepath = os.path.join(current_directory, csv_filename)
# Create the empty CSV file
with open(csv_filepath, 'w') as csv_file:
    pass  # This just creates an empty file
print("Empty CSV file created at:", csv_filepath)

# Define the list of names for the rows
row_names = ["lesion_name", "Assymentri_index", "avg_red_value", "avg_blue_value", "avg_green_value","avg_red_std_dev", "avg_blue_std_dev", "avg_red_std_dev","blue_white_veil_score","Haralick_contrast", "Haralick_dissimilarity", "Haralick_homogeneity", "Haralick_energy", "Haralick_correlation","is_cancer_bool"]

# Write the row names to the CSV file
with open(csv_filepath, 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(row_names)
print("headers added at:", csv_filepath)


Empty CSV file created at: C:\Users\elias\Desktop\Projects in data science\Final project\Attributes.csv
headers added at: C:\Users\elias\Desktop\Projects in data science\Final project\Attributes.csv


# Combine Attribute dictionaries into a single list of lists

In [35]:
#make the list
Attributelist=[]

#add values
for key in Asymmetry_results.keys():
    row=[]
    row.append(key)
    row.append(Asymmetry_results[key])
    row=row+Colour_results[key]
    row.append(Blue_white_veil_results[key])
    row=row+haralick_results[key]
    row.append(get_diagnostic(key))
    Attributelist.append(row)
    

# Add attributes to the csv

In [36]:
with open(csv_filepath, 'a', newline='') as csv_file:
    writer = csv.writer(csv_file)
    for row in Attributelist:
        writer.writerow(row)

print("Attributs appended to the CSV file at:", csv_filepath)

Attributs appended to the CSV file at: C:\Users\elias\Desktop\Projects in data science\Final project\Attributes.csv
